In [1]:
from tree import TreeNode, TreePrinter

from learn import miner
from ordinor.execution_context.rule_based.TreeNode import Node


/Users/lasse/Library/Mobile Documents/com~apple~CloudDocs/Studium Mannheim/2_Semester/Advanced Process Mining/Project/No-time-to-dice_group_project_IE692/OrdinoR-dev/ordinor/execution_context/rule_based/decision_tree.py:44: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  self._log = el[included_cols]
/Users/lasse/Library/Mobile Documents/com~apple~CloudDocs/Studium Mannheim/2_Semester/Advanced Process Mining/Project/No-time-to-dice_group_project_IE692/OrdinoR-dev/ordinor/execution_context/rule_based/decision_tree.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._log[const.RESOURCE] = self._log[const.RESOURCE].astype('category')
/Users/lasse/Library/Mobile Documents/com~ap

Decision tree initialized with an empty root node
	Dis. = 0.000000, Imp. = 1.000000, Harmonic Mean. = 0.000000, *** Tree has 1 leaf node(s). ***
Start to fit decision tree with max. height = 12
Tree grows by splitting all current leaf nodes on `tt:weekday`, step=1 (running for 00:00:11s)
	Dis. = 0.001538, Imp. = 0.999829, Harmonic Mean. = 0.003070, *** Tree has 2 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `tt:weekday`, step=2 (running for 00:00:22s)
	Dis. = 0.010478, Imp. = 0.998810, Harmonic Mean. = 0.020738, *** Tree has 3 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `at:phase`, step=3 (running for 00:00:32s)
	Dis. = 0.007995, Imp. = 0.997177, Harmonic Mean. = 0.015863, *** Tree has 5 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `at:phase`, step=4 (running for 00:00:44s)
	Dis. = 0.008072, Imp. = 0.997118, Harmonic Mean. = 0.016015, *** Tree has 6 leaf node(s). ***
Tree grows by splitting all current leaf nodes on 

In [2]:
all_nodes_matrix = []
root_node = Node(
    label=0,
    event_ids=None,
    ct_label=None,
    at_label=None,
    tt_label=None)
root_TreeNode_in_list = []
root_TreeNode_in_list = [TreeNode(root_node.label, root_node.parent_label, root_node, 0, None, [])]
all_nodes_matrix.append(root_TreeNode_in_list)
for i, step in enumerate(miner.l_history_persistent): # i to be sure we we are when navigating the tree
    all_nodes_this_step = []
    for node in step['solution'].values():
        current_node = TreeNode(node.label,node.parent_label, node, i+1, None, [])
        all_nodes_this_step.append(current_node)
    all_nodes_matrix.append(all_nodes_this_step)

print("test")

for i, all_nodes_this_step in enumerate(reversed(all_nodes_matrix)):
    for node in all_nodes_this_step:
        if node.parent_label is not None:
            print(f"{i=}, {node.parent_label=}")
            candidate_parent = next((parent_node for parent_node in all_nodes_matrix[len(
                all_nodes_matrix)-i-2] if parent_node.label == node.parent_label), None)
            if candidate_parent is not None:
                node.parent = candidate_parent
            else:
                node.parent = next((parent_node for parent_node in all_nodes_matrix[len(all_nodes_matrix)-i-2] if parent_node.label == node.label))
            if node.parent is not None:
                node.parent.children.append(node)

test
i=0, node.parent_label=1427
i=0, node.parent_label=3629
i=0, node.parent_label=3629
i=0, node.parent_label=3630
i=0, node.parent_label=3630
i=0, node.parent_label=3631
i=0, node.parent_label=3631
i=0, node.parent_label=2613
i=0, node.parent_label=2613
i=0, node.parent_label=3634
i=0, node.parent_label=3634
i=0, node.parent_label=3635
i=0, node.parent_label=3635
i=0, node.parent_label=2982
i=0, node.parent_label=2982
i=0, node.parent_label=3638
i=0, node.parent_label=3638
i=0, node.parent_label=3639
i=0, node.parent_label=3639
i=0, node.parent_label=3305
i=0, node.parent_label=3305
i=0, node.parent_label=3646
i=0, node.parent_label=3646
i=0, node.parent_label=3647
i=0, node.parent_label=3647
i=0, node.parent_label=3641
i=0, node.parent_label=3644
i=0, node.parent_label=3645
i=0, node.parent_label=4162
i=0, node.parent_label=4162
i=0, node.parent_label=4164
i=0, node.parent_label=4164
i=0, node.parent_label=4157
i=0, node.parent_label=4159
i=0, node.parent_label=4161
i=0, node.paren

In [3]:

def printAllNodesWithStep(node, step):
    if node.step > step:
        return
    elif node.step == step:
        print(node)
    if node.children:
        for child in node.children:
            printAllNodesWithStep(child, step)
    else:
        return


In [4]:
for i in range(8):
    printAllNodesWithStep(all_nodes_matrix[0][0], i)
    print("-"*150)


TreeNode [0]
------------------------------------------------------------------------------------------------------------------------------------------------------
TreeNode [339]
TreeNode [340]
------------------------------------------------------------------------------------------------------------------------------------------------------
TreeNode [339]
TreeNode [796]
TreeNode [797]
------------------------------------------------------------------------------------------------------------------------------------------------------
TreeNode [1426]
TreeNode [1427]
TreeNode [1428]
TreeNode [1429]
TreeNode [1430]
------------------------------------------------------------------------------------------------------------------------------------------------------
TreeNode [2106]
TreeNode [1427]
TreeNode [2107]
TreeNode [1429]
TreeNode [2108]
TreeNode [2109]
------------------------------------------------------------------------------------------------------------------------------------

In [5]:
treePrinter = TreePrinter(all_nodes_matrix[0][0], 7)
treePrinter.print_tree_svg()
